In [1]:
! pip install librosa numpy pandas demucs soundfile pyfluidsynth
! sudo yum install libsndfile

Package libsndfile-1.0.25-12.amzn2.1.x86_64 already installed and latest version
Nothing to do


In [2]:
import librosa
import numpy as np
import os
import pandas as pd
import io
from pathlib import Path
from shutil import rmtree
import subprocess as sp
import soundfile as sf


# Source Separation using Demucs

In [3]:
from dataclasses import dataclass

@dataclass
class DeMixedAudio4():
    sample_rate: int
    bass : np.ndarray
    drums : np.ndarray
    other : np.ndarray
    vocals : np.ndarray

@dataclass
class DeMixedAudio6():
    sample_rate: int
    bass : np.ndarray
    drums : np.ndarray
    other : np.ndarray
    vocals : np.ndarray
    piano : np.ndarray
    guitar : np.ndarray

sp.run(["python3", "-m", "demucs.separate", "-o", "../resources/tmp", "-n", model, path_to_file])- Does the model work with different sample rates? YES
- Does the model output have the same sample rate as the input? - NO: ALWAYS 44100
- If so TODO: find a way to find the input native sample rate of the audio and use this to re-load the stems

- The output volumes are different since the input audio is z score normalised by demucs

In [4]:
# file_name = os.path.basename(os.path.join(os.getcwd(),'..','resources/test_file.wav'))
# sp.run(["python3", "-m", "demucs.separate", "-o", "../resources/tmp", "-n", "htdemucs_6s", os.path.join(os.getcwd(),'..','resources/test_file.wav')])
# print("Demucs done, loading files")

In [5]:
# sp.run(["python3", "-m", "demucs.separate", "-o", "../resources/tmp", "-n", model, path_to_file])

In [6]:
def get_stems(path_to_file,model='htdemucs',output_path='../resources/tmp',delete_temp_files=True):
    file_name = os.path.basename(path_to_file)
    sp.run(["python3", "-m", "demucs.separate", "-o", output_path, "-n", model, path_to_file])
    # print("Demucs done, loading files")
    stems={}
    if model in ['htdemucs','htdemucs_ft']:
        for stem in ['bass','drums','other','vocals']:
            x,sr = librosa.load(os.path.join(output_path, model,file_name[:file_name.find('.')],f"{stem}.wav"), sr=None)
            stems[stem] = x
        if delete_temp_files:
            print("Deleting temp files")
            rmtree(f"{output_path}/{model}/{file_name[:file_name.find('.')]}")
        return DeMixedAudio4(sr, bass=stems['bass'], drums=stems['drums'], other=stems['other'], vocals=stems['vocals'])#stems
    elif model == 'htdemucs_6s':
        for stem in ['bass','drums','other','vocals','piano', 'guitar']:
            x,sr = librosa.load(os.path.join(output_path, model,file_name[:file_name.find('.')],f"{stem}.wav"), sr=None)
            stems[stem] = x
        if delete_temp_files:
            print("Deleting temp files")
            rmtree(f"{output_path}/{model}/{file_name[:file_name.find('.')]}")
        return DeMixedAudio6(sr, bass=stems['bass'], drums=stems['drums'], other=stems['other'], vocals=stems['vocals'], piano=stems['piano'], guitar=stems['guitar'])#stems

In [7]:
songname='WhenTheLeveeBreaks'
song = get_stems(os.path.join(os.getcwd(),'..',f'resources/{songname}.wav'), model='htdemucs_6s', delete_temp_files=False)

Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/5c90dfd2-34c22ccb.th" to /home/ec2-user/.cache/torch/hub/checkpoints/5c90dfd2-34c22ccb.th
100%|██████████| 52.4M/52.4M [00:01<00:00, 33.9MB/s]
ffprobe: error while loading shared libraries: libopenh264.so.5: cannot open shared object file: No such file or directory


Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/ec2-user/SageMaker/scalable-asset-generation/resources/tmp/htdemucs_6s
Separating track /home/ec2-user/SageMaker/scalable-asset-generation/src/../resources/WhenTheLeveeBreaks.wav


100%|████████████████████████████████████████████████████████████████████████| 432.9/432.9 [01:02<00:00,  6.97seconds/s]


In [8]:
song

DeMixedAudio6(sample_rate=44100, bass=array([1.5258789e-05, 1.5258789e-05, 1.5258789e-05, ..., 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00], dtype=float32), drums=array([-9.1552734e-05, -9.1552734e-05, -1.0681152e-04, ...,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00], dtype=float32), other=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), vocals=array([1.5258789e-05, 3.0517578e-05, 3.0517578e-05, ..., 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00], dtype=float32), piano=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), guitar=array([1.5258789e-05, 1.5258789e-05, 1.5258789e-05, ..., 1.5258789e-05,
       1.5258789e-05, 3.0517578e-05], dtype=float32))

In [33]:
melodic_line = song.guitar + song.bass+ song.other + song.piano
sf.write(os.path.join(os.getcwd(), '..',f'resources/{songname}_melodicline.wav'),melodic_line, song.sample_rate)

vocal = song.vocals
sf.write(os.path.join(os.getcwd(), '..',f'resources/{songname}_vocal.wav'),vocal, song.sample_rate)

# Pop2Piano

In [ ]:
!apt-get install -y fluidsynth
!sudo yum install fluidsynth
!git clone https://github.com/sweetcocoa/pop2piano/
!cd pop2piano
!pip install pretty-midi==0.2.9 omegaconf==2.1.1 youtube-dl==2021.12.17 transformers==4.16.1 pytorch-lightning==1.8.4 essentia==2.1b6.dev609 note-seq==0.0.3 pyFluidSynth==1.3.0
!wget https://github.com/sweetcocoa/pop2piano/releases/download/dpi_2k_epoch/model-1999-val_0.67311615.ckpt

In [10]:
import os 
import sys
sys.path.append("pop2piano")
import glob
import random

import torch
import torchaudio
import librosa
import numpy as np
import pandas as pd
import IPython.display as ipd
import soundfile as sf
# from google.colab import files

from tqdm.auto import tqdm
from omegaconf import OmegaConf
import note_seq

from utils.dsp import get_stereo
from utils.demo import download_youtube
from transformer_wrapper import TransformerWrapper
from midi_tokenizer import MidiTokenizer, extrapolate_beat_times
from preprocess.beat_quantizer import extract_rhythm, interpolate_beat_times

from mido import MidiFile

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


In [ ]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# config = OmegaConf.load("pop2piano/config.yaml")
# wrapper = TransformerWrapper(config)
# wrapper = wrapper.load_from_checkpoint("model-1999-val_0.67311615.ckpt", config=config).to(device)
# model = "dpipqxiy"

# composer='composer1'
# wrapper.eval()

In [ ]:
# pm, composer, mix_path, midi_path = wrapper.generate(
#     audio_path=os.path.join(os.getcwd(),'..','resources/test_file.wav'), 
#     composer=composer, 
#     model=model,
#     show_plot=False, 
#     save_midi=True, 
#     save_mix=False, 
# )
# # note_seq.plot_sequence(note_seq.midi_to_note_sequence(pm))

In [11]:
def get_piano(path_to_file, composer='composer1', delete_temp_files=True):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    config = OmegaConf.load("pop2piano/config.yaml")
    wrapper = TransformerWrapper(config)
    wrapper = wrapper.load_from_checkpoint("model-1999-val_0.67311615.ckpt", config=config).to(device)
    model = "dpipqxiy"

    wrapper.eval()
    pm, composer, mix_path, midi_path = wrapper.generate(
        audio_path=path_to_file, 
        composer=composer, 
        model=model,
        show_plot=False, 
        save_midi=True, 
        save_mix=False, 
    )
    midi_file = MidiFile(midi_path)
    if delete_temp_files:
        os.remove(midi_path)
    
    return midi_file

In [35]:
_ = get_piano(f'../resources/{songname}.wav',composer='composer1', delete_temp_files=False)
_ = get_piano(f'../resources/{songname}_melodicline.wav',composer='composer1', delete_temp_files=False)
_ = get_piano(f'../resources/{songname}_vocal.wav',composer='composer1', delete_temp_files=False)

In [36]:
# midi = get_piano('../resources/test_resources/test_file-48000.wav')
midi = get_piano(f'../resources/{songname}_vocal.wav',composer='composer1', delete_temp_files=False)
# midi

In [81]:
type(midi)

mido.midifiles.midifiles.MidiFile

# Output

In [ ]:
acapella = song.vocals

In [ ]:
acapella

# Stem gain testing

In [ ]:
!pip install pyloudnorm

In [ ]:
import pyloudnorm as pyln

In [ ]:
meter = pyln.Meter(song.sample_rate) # create BS.1770 meter
meter.integrated_loudness(song.vocals) # measure loudness

In [ ]:
# Original track loudness
meter.integrated_loudness(librosa.load(os.path.join(os.getcwd(),'..','resources/test_file.wav'))[0])

In [ ]:
# Reconstructed track loudness
meter.integrated_loudness(song.vocals + song.other + song.bass + song.drums)

In [ ]:
x,sr = librosa.load(os.path.join(os.getcwd(),'..','resources/test_file.wav'), sr=None)

x_48k = librosa.resample(x, orig_sr=sr, target_sr=48000)

In [ ]:
import soundfile as sf
sf.write('../resources/test_resources/test_file-48000.wav',x_48k, 48000)

In [ ]:
x_test,sr_test = librosa.load('../resources/test_resources/test_file-48000.wav', sr=None)

In [ ]:
sr_test

In [ ]:
song = get_stems(os.path.join(os.getcwd(),'..','resources/test_resources/test_file-48000.wav'))

Humanization- https://github.com/erwald/rachel

# Processing Pipeline

In [12]:
import mido
@dataclass
class ProcessedSong4():
    stems:DeMixedAudio4
    vocal_based_midi: mido.midifiles.midifiles.MidiFile
    melody_based_midi:mido.midifiles.midifiles.MidiFile
    full_song_based_midi:mido.midifiles.midifiles.MidiFile
    
@dataclass
class ProcessedSong6():
    stems:DeMixedAudio6
    vocal_based_midi: mido.midifiles.midifiles.MidiFile
    melody_based_midi:mido.midifiles.midifiles.MidiFile
    full_song_based_midi:mido.midifiles.midifiles.MidiFile
    

In [16]:
def process_file(audio_file:np.ndarray, sample_rate:int, song_name:str, demucs_model='htdemucs_6s', output_folder_path:str='../resources/tmp'):
    if not os.path.exists(output_folder_path):
        os.mkdir(output_folder_path)
    
    # Write array to audio file for demucs and pop2piano
    sf.write(os.path.join(output_folder_path,f'{song_name}.wav'), audio_file, sample_rate)
    # if not os.path.exists(os.path.join(output_folder_path, demucs_model, song_name)):
    # Separate stems using demucs
    print("Running demucs")
    song = get_stems(path_to_file=os.path.join(output_folder_path,f'{song_name}.wav'), model=demucs_model,output_path=output_folder_path, delete_temp_files=False)
    # Save the melodic line (guitar+bass+other+piano)
    if demucs_model == 'htdemucs_6s':
        melodic_line = song.guitar + song.bass+ song.other + song.piano
    if demucs_model in ['htdemucs','htdemucs_ft']:
        melodic_line = song.other+song.bass
    sf.write(os.path.join(output_folder_path,f'{song_name}_melodicline.wav'),melodic_line, song.sample_rate)
    vocal = song.vocals
    sf.write(os.path.join(output_folder_path,f'{song_name}_vocal.wav'),vocal, song.sample_rate)

    # Run pop2piano on full song, melodic line and vocal only
    print("Running pop2piano")
    full_song_midi = get_piano(os.path.join(output_folder_path,f'{song_name}.wav'),composer='composer1', delete_temp_files=False)
    melodicline_midi = get_piano(os.path.join(output_folder_path,f'{song_name}_melodicline.wav'),composer='composer1', delete_temp_files=False)
    vocal_only_midi = get_piano(os.path.join(output_folder_path,f'{song_name}_vocal.wav'),composer='composer1', delete_temp_files=False)
    
    if demucs_model == 'htdemucs_6s':
        return ProcessedSong6(song, vocal_only_midi, melodicline_midi, full_song_midi)
    if demucs_model in ['htdemucs','htdemucs_ft']:
        return ProcessedSong4(song, vocal_only_midi, melodicline_midi, full_song_midi)
    

In [14]:
x_test,sr_test = librosa.load('../resources/TakeOnMe.wav', sr=None)

In [15]:
f = process_file(x_test, sr_test, song_name='TakeOnMe',demucs_model='htdemucs_ft',output_folder_path='../resources/tmp/test')

Running demucs


Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/f7e0c4bc-ba3fe64a.th" to /home/ec2-user/.cache/torch/hub/checkpoints/f7e0c4bc-ba3fe64a.th
100%|██████████| 80.2M/80.2M [00:02<00:00, 40.2MB/s]
Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/d12395a8-e57c48e6.th" to /home/ec2-user/.cache/torch/hub/checkpoints/d12395a8-e57c48e6.th
100%|██████████| 80.2M/80.2M [00:02<00:00, 38.1MB/s]
Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/92cfc3b6-ef3bcb9c.th" to /home/ec2-user/.cache/torch/hub/checkpoints/92cfc3b6-ef3bcb9c.th
100%|██████████| 80.2M/80.2M [00:02<00:00, 36.1MB/s]
Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/04573f0d-f3cf25b2.th" to /home/ec2-user/.cache/torch/hub/checkpoints/04573f0d-f3cf25b2.th
100%|██████████| 80.2M/80.2M [00:02<00:00, 36.6MB/s]
ffprobe: error while loading shared libraries: libopenh264.so.5: cannot open shared object file: No such file or directory


Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/ec2-user/SageMaker/scalable-asset-generation/resources/tmp/test/htdemucs_ft
Separating track ../resources/tmp/test/TakeOnMe.wav


100%|████████████████████████████████████████████████████████████████████████| 245.7/245.7 [00:11<00:00, 21.79seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 245.7/245.7 [00:09<00:00, 24.65seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 245.7/245.7 [00:09<00:00, 24.68seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 245.7/245.7 [00:09<00:00, 24.64seconds/s]


Running pop2piano


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

In [17]:
f

ProcessedSong4(stems=DeMixedAudio4(sample_rate=44100, bass=array([0.0000000e+00, 1.5258789e-05, 0.0000000e+00, ..., 3.0517578e-05,
       3.0517578e-05, 3.0517578e-05], dtype=float32), drums=array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 3.0517578e-05,
       3.0517578e-05, 3.0517578e-05], dtype=float32), other=array([9.1552734e-05, 1.5258789e-05, 3.0517578e-05, ..., 3.0517578e-05,
       1.5258789e-05, 1.5258789e-05], dtype=float32), vocals=array([ 4.5776367e-05, -1.5258789e-05, -4.5776367e-05, ...,
        3.0517578e-05,  3.0517578e-05,  3.0517578e-05], dtype=float32)), vocal_based_midi=MidiFile(type=1, ticks_per_beat=384, tracks=[
  MidiTrack([
    MetaMessage('set_tempo', tempo=500000, time=0),
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('end_of_track', time=1)]),
  MidiTrack([
    Message('program_change', channel=0, program=0, time=0),
    Message('note_on', channel=0, note=74